In [8]:
import os
import uuid
import numpy as np
import pandas as pd
import itertools

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
from isadream import demos, demo_path, io
from isadream.models import utils
from isadream.display import helpers
from isadream import io

In [5]:
import bokeh as bk
import bokeh.models
import bokeh.layouts
import bokeh.palettes
import bokeh.plotting
import bokeh.transform

In [11]:
NMR_SIM_GROUPS = dict(
    x_groups=(
        ('Al-Na Distance', ('Al-M Distance',), ("Al",)),
        ('Periodic Box Size', ('Periodic Box Size',), ("Al")),
        ('Solvent Molecule Count', ('Solvent Molecule Count',), ("Al")),
    ),
    y_groups=(
        ('Isotropic Shielding Factor', ('Isotropic Shielding Factor',), ("Al")),
    ),
    dx_groups=(
        # ---
        # Molality calculation.
        #   = number of moles of solute per kilogram of solvent.
        # 
        # Calculate the kg of the solvent.
        #  1 atomic mass unit / (angstrom^3) = 1 660.53904 kg / m3
        #
        # ---
        (
            "Estimated Molality",
            ("Solvent Molecule Count", "Periodic Box Size"),
            (
                lambda x, y: 1 / ( ( (np.array(x) * 18.02) / np.array(y)**3 ) / 16.6053904 )

            )
        ),
    ),
    dy_groups=(
        (
            "ppm delta",
            # Isotropic Calculation
            # intercept - sigma / - slope
            ("Isotropic Shielding Factor", ), 
            (lambda x: np.array(x) - 532.36)
        ),
    )
)

In [17]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["ERNESTO_NMR_1"]))
nmr_nodes = io.parse_node_json(demo_json)
sim_df, sim_md_df, sim_md = io.prepare_nodes_for_bokeh(NMR_SIM_GROUPS["x_groups"], NMR_SIM_GROUPS["y_groups"], [nmr_nodes])

sim_df, sim_md_df, sim_md = io.prepare_nodes_for_bokeh(
    NMR_SIM_GROUPS["x_groups"],
    NMR_SIM_GROUPS["y_groups"],
    [nmr_nodes])

# Create any requested derived columns.
if NMR_SIM_GROUPS.get("dx_groups"):
    for group in NMR_SIM_GROUPS.get("dx_groups"):
        sim_df = helpers.create_derived_column(sim_df, group)

if NMR_SIM_GROUPS.get("dy_groups"):
    for group in NMR_SIM_GROUPS.get("dy_groups"):
        sim_df = helpers.create_derived_column(sim_df, group)

In [20]:
def cat_dff_panel(x_groups,
                y_groups,
                main_df: pd.DataFrame,
                metadata_df: pd.DataFrame,
                metadata: dict):
    """

    :param x_groups:
    :param y_groups:
    :param data:
    :param metadata:
    :return:

    """
    source = bk.models.ColumnDataSource(main_df)

    x_keys = helpers.get_group_keys(x_groups)
    y_keys = helpers.get_group_keys(y_groups)

    p = bk.plotting.figure(y_range="ppm delta", plot_width=900, 
                           x_range=(-15, 15), toolbar_location=None)
    
    for key, val in pd.groupby(main_df, "ppm delta"):
        p.patch(key, val, alpha=0.6, line_color="black")

    p.outline_line_color = None
    p.background_fill_color = "#efefef"

    p.xaxis.ticker = bk.models.FixedTicker(ticks=list(range(0, 101, 10)))
    p.xaxis.formatter = bk.models.PrintfTickFormatter(format="%d%%")

    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = "#dddddd"
    p.xgrid.ticker = p.xaxis[0].ticker

    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None

    p.y_range.range_padding = 0.12
    
    

#     panel = bk.models.Panel(child=table, title=TITLE)

    return p

In [22]:
f = cat_dff_panel(NMR_SIM_GROUPS["x_groups"],
    NMR_SIM_GROUPS["y_groups"], sim_df, sim_md_df, sim_md)

f

/home/tyler/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: pd.groupby() is deprecated and will be removed; Please use the Series.groupby() or DataFrame.groupby() methods


bokeh.plotting.figure.Figure(
    id='95747815-8e89-48c5-91be-edf3ab7ae626',
    above=[],
    aspect_scale=1,
    background_fill_alpha={'value': 1.0},
    background_fill_color={'value': '#efefef'},
    below=[bokeh.models.axes.LinearAxis(
         id='7e4dacde-60ad-4f0b-b598-2307c2a20619',
         axis_label='',
         axis_label_standoff=5,
         axis_label_text_align='left',
         axis_label_text_alpha={'value': 1.0},
         axis_label_text_baseline='bottom',
         axis_label_text_color={'value': '#444444'},
         axis_label_text_font='helvetica',
         axis_label_text_font_size={'value': '10pt'},
         axis_label_text_font_style='italic',
         axis_label_text_line_height=1.2,
         axis_line_alpha={'value': 1.0},
         axis_line_cap='butt',
         axis_line_color={'value': None},
         axis_line_dash=[],
         axis_line_dash_offset=0,
         axis_line_join='bevel',
         axis_line_width={'value': 1},
         bounds='auto',
         fixed_location=None,
         formatter=bokeh.models.formatters.PrintfTickFormatter(
             id='a65cd4ff-6e36-47ff-a5d3-bcde7806cc0b',
             format='%d%%',
             js_event_callbacks={},
             js_property_callbacks={},
             name=None,
             subscribed_events=[],
             tags=[]),
         js_event_callbacks={},
         js_property_callbacks={},
         level='overlay',
         major_label_orientation='horizontal',
         major_label_overrides={},
         major_label_standoff=5,
         major_label_text_align='center',
         major_label_text_alpha={'value': 1.0},
         major_label_text_baseline='alphabetic',
         major_label_text_color={'value': '#444444'},
         major_label_text_font='helvetica',
         major_label_text_font_size={'value': '8pt'},
         major_label_text_font_style='normal',
         major_label_text_line_height=1.2,
         major_tick_in=2,
         major_tick_line_alpha={'value': 1.0},
         major_tick_line_cap='butt',
         major_tick_line_color={'value': None},
         major_tick_line_dash=[],
         major_tick_line_dash_offset=0,
         major_tick_line_join='bevel',
         major_tick_line_width={'value': 1},
         major_tick_out=6,
         minor_tick_in=0,
         minor_tick_line_alpha={'value': 1.0},
         minor_tick_line_cap='butt',
         minor_tick_line_color={'value': None},
         minor_tick_line_dash=[],
         minor_tick_line_dash_offset=0,
         minor_tick_line_join='bevel',
         minor_tick_line_width={'value': 1},
         minor_tick_out=4,
         name=None,
         plot=bokeh.plotting.figure.Figure(id='95747815-8e89-48c5-91be-edf3ab7ae626', ...),
         subscribed_events=[],
         tags=[],
         ticker=bokeh.models.tickers.FixedTicker(
             id='c86cd230-d091-4209-82ee-1707d3be9801',
             desired_num_ticks=6,
             js_event_callbacks={},
             js_property_callbacks={},
             name=None,
             num_minor_ticks=5,
             subscribed_events=[],
             tags=[],
             ticks=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]),
         visible=True,
         x_range_name='default',
         y_range_name='default')],
    border_fill_alpha={'value': 1.0},
    border_fill_color={'value': '#ffffff'},
    css_classes=[],
    disabled=False,
    extra_x_ranges={},
    extra_y_ranges={},
    h_symmetry=True,
    height=None,
    hidpi=True,
    js_event_callbacks={},
    js_property_callbacks={},
    left=[bokeh.models.axes.CategoricalAxis(
         id='fda5d4d7-c5d7-4fc7-9fa7-a493c8c7ce45',
         axis_label='',
         axis_label_standoff=5,
         axis_label_text_align='left',
         axis_label_text_alpha={'value': 1.0},
         axis_label_text_baseline='bottom',
         axis_label_text_color={'value': '#444444'},
         axis_label_text_font='helvetica',
         axis_label_text_font_size={'value': '10pt'},
         axis_label_text_font_style='itali

In [7]:
sim_df.head()

,Al-Na Distance,Periodic Box Size,Solvent Molecule Count,Isotropic Shielding Factor,Estimated Molality,ppm delta
0,4.0,14.35,90,535.5807,30.25572,3.2207
1,4.0,14.35,90,539.3779,30.25572,7.0179
2,4.0,14.35,90,531.1274,30.25572,-1.2326
3,4.0,14.35,90,527.5177,30.25572,-4.8423
4,4.0,14.35,90,534.3948,30.25572,2.0348


In [8]:
p = hv.Points(sim_df, ["Isotropic Shielding Factor", "ppm delta"])
p

:Points   [Isotropic Shielding Factor,ppm delta]   (Al-Na Distance,Periodic Box Size,Solvent Molecule Count,Estimated Molality)

In [2]:
# demo_json = io.read_idream_json(os.path.join(demo_path, demos["ERNESTO_NMR_1"]))
# node = io.parse_node_json(demo_json)

In [3]:
# main_df, metadata_df, metadata = io.prepare_nodes_for_bokeh(x_groups, y_groups, [node])

In [18]:
# main_df

,Al-Na Distance,Periodic Box Size,Solvent Molecule Count,Isotropic Shielding Factor
0,4.0,14.35,90,535.5807
1,4.0,14.35,90,539.3779
2,4.0,14.35,90,531.1274
3,4.0,14.35,90,527.5177
4,4.0,14.35,90,534.3948
5,4.0,14.35,90,524.4831
6,4.0,14.35,90,524.0155
7,4.0,14.35,90,533.2804
8,4.0,14.35,90,527.3082
9,4.0,14.35,90,521.0975


In [4]:
x_groups=(('Total Aluminate Concentration', ('Molar',), ("Al",)),
          ('Counter Ion Concentration', ('Molar',),
           ("Na+", "Li+", "Cs+", "K+")),
          ('Counter Ion', ('Species',), ("Na+", "Li+", "Cs+", "K+",)),
          ('Base Concentration', ('Molar',), ("OH-",)))

y_groups=(('27 Al ppm', ('ppm',), ("Al",)),)

test_groups = x_groups + y_groups

In [5]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR_V2"]))
node = io.parse_node_json(demo_json)

In [6]:
json_paths = [os.path.join(demo_path, demo)
              for demo in [demos["SIPOS_NMR"], demos["SIPOS_NMR_V2"]]]

nodes = io.create_drupal_nodes(json_paths)
main_df, metadata_df, metadata = io.prepare_nodes_for_bokeh(x_groups, y_groups, nodes)

In [7]:
main_df

,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration,27 Al ppm
0,0.005,2.93,K+,2.93,79.90
1,0.005,4.92,K+,4.92,79.84
2,0.005,6.85,K+,6.85,79.72
3,0.005,9.13,K+,9.13,79.66
4,0.005,10.71,K+,10.71,79.66
5,0.005,0.66,Li+,0.66,79.92
6,0.005,1.10,Li+,1.10,79.84
7,0.005,1.64,Li+,1.64,79.66
8,0.005,2.14,Li+,2.14,79.54
9,0.005,2.59,Li+,2.59,79.36


In [8]:
data, metadata = io.collate_node(node, test_groups)
# display(data)
# type(data)
# display(data[0])
# data[("Counter Ion Concentration")]
# data.iloc[[0, 1], :]
data
# data.swaplevel(0,1, axis=1).xs("data", axis=1)
# data.swaplevel(0,1, axis=1).xs("metadata", axis=1).iloc[[0, 1], :]

Total Aluminate Concentration  \
                            data   
0                          0.050   
1                          0.050   
2                          0.050   
3                          0.050   
4                          0.050   
5                          0.050   
6                          0.050   
7                          0.050   
8                          0.050   
9                          0.050   
10                         0.050   
11                         0.100   
12                         0.100   
13                         0.100   
14                         0.100   
15                         0.100   
16                         0.100   
17                         0.100   
18                         0.100   
19                         0.100   
20                         0.100   
21                         0.100   
22                         0.396   
23                         0.396   
24                         0.396   
25                         0.396   
26                         0.396   
27                         0.396   
28                         0.396   
29                         0.396   
..                           ...   
39                         0.800   
40                         0.800   
41                         0.800   
42                         1.200   
43                         1.200   
44                         1.200   
45                         1.200   
46                         1.200   
47                         1.200   
48                         1.200   
49                         1.200   
50                         1.200   
51                         1.200   
52                         1.200   
53                         2.000   
54                         2.000   
55                         2.000   
56                         2.000   
57                         2.000   
58                         2.000   
59                         2.000   
60                         2.000   
61                         2.000   
62                         3.000   
63                         3.000   
64                         3.000   
65                         3.000   
66                         3.000   
67                         3.000   
68                         3.000   

                                                       \
                                             metadata   
0   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
1   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
2   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
3   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
4   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
5   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
6   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
7   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
8   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
9   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
10  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
11  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
12  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
13  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
14  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
15  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
16  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
17  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
18  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
19  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
20  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
21  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
22  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
23  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
24  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
25  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
26  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
27  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
28  (91746262-7cd3-37ac-